<a href="https://colab.research.google.com/github/gwanghuiHa/Toyboxes/blob/main/Toybox_QuadOptics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import (
    VBox, HBox, FloatSlider, FloatLogSlider, FloatText, Dropdown,
    Button, RadioButtons, Layout, Output
)
from IPython.display import display, clear_output

IN_NOTEBOOK = True
LQ_EFFECTIVE = 0.10  # 10 cm quad length

# ----------------------------
# Beam optics helpers
# ----------------------------

def drift_matrix(L):
    return np.array([[1.0, L],
                     [0.0, 1.0]])

def gradient_to_K(G_Tperm, energy_MeV):
    """K [1/m^2] from gradient G [T/m] and beam energy [MeV]."""
    e = 1.602176634e-19
    m_e = 9.10938356e-31
    c = 299792458.0
    mc2_MeV = 0.511

    E_tot_MeV = energy_MeV + mc2_MeV
    gamma = E_tot_MeV / mc2_MeV
    beta = np.sqrt(1.0 - 1.0 / gamma**2)
    p = beta * gamma * m_e * c
    return e * G_Tperm / p   # [1/m^2]

def quad_thick_matrix(L, K_plane):
    """Thick-lens quad matrix for one plane, length L, K_plane [1/m^2]."""
    if np.isclose(K_plane, 0.0):
        return drift_matrix(L)

    if K_plane > 0:
        k = np.sqrt(K_plane)
        phi = k * L
        c = np.cos(phi)
        s = np.sin(phi)
        return np.array([[c,      s / k],
                         [-k * s, c    ]])
    else:
        k = np.sqrt(-K_plane)
        phi = k * L
        c = np.cosh(phi)
        s = np.sinh(phi)
        return np.array([[c,      s / k],
                         [k * s,  c    ]])

def build_lattice(quads_with_K, z_max):
    """
    Build lattice with thick quads and drifts, return segments + z_grid.
    quads_with_K: [{'z': center, 'K': K}, ...]
    """
    quads_sorted = sorted(quads_with_K, key=lambda q: q['z'])
    segments = []
    z_prev = 0.0

    for q in quads_sorted:
        zc = q['z']; K = q['K']
        z_enter = zc - LQ_EFFECTIVE / 2.0
        z_exit  = zc + LQ_EFFECTIVE / 2.0

        if z_enter > z_prev:
            segments.append({'type': 'drift', 'L': z_enter - z_prev})
            z_prev = z_enter

        segments.append({'type': 'quad', 'L': LQ_EFFECTIVE, 'K': K})
        z_prev = z_exit

    if z_prev < z_max:
        segments.append({'type': 'drift', 'L': z_max - z_prev})

    # Coarser grid to speed envelope drawing
    z_grid = np.linspace(0.0, z_max, 100)
    return segments, z_grid

def propagate_sigma_plane(segments, z_grid, Sigma0, signK=+1):
    """
    Propagate 2x2 sigma matrix through lattice for one plane.
    signK = +1 for x, -1 for y (x sees +K, y sees -K).
    """
    z_current = 0.0
    M = np.eye(2)
    Sigma_list = []

    if len(segments) == 0:
        segments = [{'type': 'drift', 'L': max(z_grid[-1], 1e-6)}]

    seg_index = 0
    seg = segments[seg_index]
    seg_type = seg['type']
    seg_remaining = seg['L']
    seg_K = seg.get('K', 0.0)

    for z in z_grid:
        dz_total = z - z_current
        while dz_total > 1e-12:
            step = min(dz_total, seg_remaining)

            if seg_type == 'drift':
                M_step = drift_matrix(step)
            else:
                K_plane = signK * seg_K
                M_step = quad_thick_matrix(step, K_plane)

            M = M_step @ M
            z_current += step
            dz_total  -= step
            seg_remaining -= step

            if seg_remaining <= 1e-12:
                seg_index += 1
                if seg_index < len(segments):
                    seg = segments[seg_index]
                    seg_type = seg['type']
                    seg_remaining = seg['L']
                    seg_K = seg.get('K', 0.0)
                else:
                    seg_type = 'drift'
                    seg_remaining = 1e9
                    seg_K = 0.0

        Sigma = M @ Sigma0 @ M.T
        Sigma_list.append(Sigma)

    Sigma_array = np.stack(Sigma_list, axis=0)
    sigma_pos = np.sqrt(Sigma_array[:, 0, 0])
    return sigma_pos, Sigma_array

def sigma_from_particles(x, xp):
    X = np.vstack((x, xp))
    return np.cov(X)

def generate_initial_distribution(N, params):
    sigx = params['sigx']; sigy = params['sigy']
    sigxp = params['sigxp']; sigyp = params['sigyp']
    slope_x = params['slope_x']; slope_y = params['slope_y']
    dist_type = params['dist_type']

    if dist_type == 'Gaussian':
        x = np.random.normal(0.0, sigx, N)
        y = np.random.normal(0.0, sigy, N)
    else:
        R = 2.0 * max(sigx, sigy)
        r = R * np.sqrt(np.random.rand(N))
        theta = 2 * np.pi * np.random.rand(N)
        x = r * np.cos(theta)
        y = r * np.sin(theta)

    xp_unc = np.random.normal(0.0, sigxp, N)
    yp_unc = np.random.normal(0.0, sigyp, N)
    xp = slope_x * x + xp_unc
    yp = slope_y * y + yp_unc
    return x, xp, y, yp

def propagate_particles_to_z(x0, xp0, y0, yp0, segments, z_target):
    """
    Propagate particles to z_target using same segments.
    x sees +K, y sees -K (positive G -> x focus, y defocus).
    """
    z_current = 0.0
    Mx = np.eye(2); My = np.eye(2)

    if len(segments) == 0:
        segments = [{'type': 'drift', 'L': z_target}]

    seg_index = 0
    seg = segments[seg_index]
    seg_type = seg['type']
    seg_remaining = seg['L']
    seg_K = seg.get('K', 0.0)

    z_target = float(z_target)

    while z_current < z_target - 1e-12:
        dz_total = z_target - z_current
        step = min(dz_total, seg_remaining)

        if seg_type == 'drift':
            M_step = drift_matrix(step)
            Mx = M_step @ Mx; My = M_step @ My
        else:
            Kx = +seg_K; Ky = -seg_K
            Mx_step = quad_thick_matrix(step, Kx)
            My_step = quad_thick_matrix(step, Ky)
            Mx = Mx_step @ Mx; My = My_step @ My

        z_current += step
        seg_remaining -= step

        if seg_remaining <= 1e-12:
            seg_index += 1
            if seg_index < len(segments):
                seg = segments[seg_index]
                seg_type = seg['type']
                seg_remaining = seg['L']
                seg_K = seg.get('K', 0.0)
            else:
                seg_type = 'drift'
                seg_remaining = 1e9
                seg_K = 0.0

    X0 = np.vstack((x0, xp0))
    Y0 = np.vstack((y0, yp0))
    Xz = Mx @ X0; Yz = My @ Y0
    return Xz[0, :], Xz[1, :], Yz[0, :], Yz[1, :]

# ----------------------------
# Plot helpers
# ----------------------------

def create_initial_distribution_figure():
    fig, axs = plt.subplots(2, 2, figsize=(7.5, 5.5))
    ax_xxprime = axs[0, 1]
    ax_yyprime = axs[1, 0]
    ax_xy      = axs[1, 1]
    axs[0, 0].axis('off')
    ax_xxprime.set_xlabel("x [mm]"); ax_xxprime.set_ylabel("x' [mrad]")
    ax_yyprime.set_xlabel("y [mm]"); ax_yyprime.set_ylabel("y' [mrad]")
    ax_xy.set_xlabel("x [mm]");      ax_xy.set_ylabel("y [mm]")
    fig.tight_layout()
    return fig, axs

def update_initial_distribution_plots(
        axs, x, xp, y, yp,
        xrange_mm, yrange_mm, xprange_mrad, yprange_mrad):
    ax_xxprime = axs[0, 1]
    ax_yyprime = axs[1, 0]
    ax_xy      = axs[1, 1]

    for ax in [ax_xxprime, ax_yyprime, ax_xy]:
        ax.cla()

    x_mm = x * 1e3; y_mm = y * 1e3
    xp_mrad = xp * 1e3; yp_mrad = yp * 1e3

    ax_xxprime.scatter(x_mm, xp_mrad, s=2, alpha=0.4)
    ax_xxprime.set_xlabel("x [mm]"); ax_xxprime.set_ylabel("x' [mrad]")
    ax_xxprime.set_xlim(-xrange_mm, xrange_mm)
    ax_xxprime.set_ylim(-xprange_mrad, xprange_mrad)

    ax_yyprime.scatter(y_mm, yp_mrad, s=2, alpha=0.4)
    ax_yyprime.set_xlabel("y [mm]"); ax_yyprime.set_ylabel("y' [mrad]")
    ax_yyprime.set_xlim(-yrange_mm, yrange_mm)
    ax_yyprime.set_ylim(-yprange_mrad, yprange_mrad)

    ax_xy.scatter(x_mm, y_mm, s=2, alpha=0.4)
    ax_xy.set_xlabel("x [mm]"); ax_xy.set_ylabel("y [mm]")
    ax_xy.set_xlim(-xrange_mm, xrange_mm)
    ax_xy.set_ylim(-yrange_mm, yrange_mm)

    for ax in [ax_xxprime, ax_yyprime, ax_xy]:
        ax.grid(True)

def create_beamline_figure():
    fig, (ax_envelope, ax_cross) = plt.subplots(
        1, 2, figsize=(10, 4.5),
        gridspec_kw={'width_ratios': [2, 1]}
    )
    ax_envelope.set_xlabel("z [m]")
    ax_envelope.set_ylabel("Beam size [mm]")
    ax_cross.set_xlabel("x [mm]")
    ax_cross.set_ylabel("y [mm]")
    fig.tight_layout()
    return fig, ax_envelope, ax_cross

def draw_envelope_and_quads(
        ax_envelope, z_grid, sigx_z, sigy_z,
        quads, z_max, selected_index,
        z_view=None, yscale_factor=1.0):

    ax_envelope.cla()
    sigx_mm = sigx_z * 1e3
    sigy_mm = sigy_z * 1e3
    ax_envelope.plot(z_grid, sigx_mm, label='σx')
    ax_envelope.plot(z_grid, sigy_mm, label='σy')

    current_max = max(np.max(sigx_mm), np.max(sigy_mm)) * 1.1
    y_max = current_max * yscale_factor if current_max > 0 else 1.0
    ax_envelope.set_ylim(0, y_max)
    ax_envelope.set_xlim(0, z_max)

    quad_height = 0.15 * y_max
    for i, q in enumerate(quads):
        zc = q['z']
        left = zc - LQ_EFFECTIVE / 2.0
        width = LQ_EFFECTIVE
        color = 'tab:blue' if i != selected_index else 'tab:orange'
        ax_envelope.add_patch(
            plt.Rectangle((left, 0.0), width, quad_height,
                          color=color, alpha=0.8, zorder=0)
        )
        ax_envelope.text(
            zc, quad_height + 0.03 * y_max,
            f"Q{i}", ha='center', va='bottom', fontsize=8
        )

    if z_view is not None:
        ax_envelope.axvline(z_view, color='red', linestyle='--', alpha=0.7)

    ax_envelope.set_xlabel("z [m]")
    ax_envelope.set_ylabel("Beam size [mm]")
    ax_envelope.legend()
    ax_envelope.grid(True)

def draw_cross_section(ax_cross, xz, yz, xrange_mm, yrange_mm):
    ax_cross.cla()
    x_mm = xz * 1e3; y_mm = yz * 1e3
    ax_cross.scatter(x_mm, y_mm, s=3, alpha=0.4)
    ax_cross.set_xlabel("x [mm]"); ax_cross.set_ylabel("y [mm]")
    ax_cross.set_xlim(-xrange_mm, xrange_mm)
    ax_cross.set_ylim(-yrange_mm, yrange_mm)
    ax_cross.grid(True)
    ax_cross.set_aspect('equal', 'box')

# ----------------------------
# Main interactive function
# ----------------------------

def launch_toybox():
    clear_output(wait=True)

    N_phys = 800          # particles used for physics / statistics
    N_plot_xsec = 500     # max particles plotted in x-y cross-section

    init_params = {
        'dist_type': 'Gaussian',
        'sigx': 1e-3, 'sigy': 1e-3,
        'sigxp': 1e-4, 'sigyp': 1e-4,
        'slope_x': 0.0, 'slope_y': 0.0,
    }

    quads = [{'z': 1.0, 'G': 1.0}]   # one quad at 1 m, 1 T/m
    z_max_default = 10.0
    energy_default = 50.0  # MeV

    fig_init, axs_init = create_initial_distribution_figure()
    fig_beam, ax_env, ax_cross = create_beamline_figure()

    # Avoid stray static copies in Colab
    plt.close(fig_init)
    plt.close(fig_beam)

    out_init = Output()
    out_beam = Output()
    with out_init: display(fig_init)
    with out_beam: display(fig_beam)

    # ---- widgets ----
    dist_type_w = RadioButtons(
        options=['Gaussian', 'Uniform circular'],
        value='Gaussian', description='XY shape:',
        layout=Layout(width='200px')
    )

    # Initial params: heavy → continuous_update=False
    sigx_w  = FloatLogSlider(value=1e-3, base=10, min=-4, max=-2,
                             step=0.01, description='σx [m]:',
                             continuous_update=False)
    sigy_w  = FloatLogSlider(value=1e-3, base=10, min=-4, max=-2,
                             step=0.01, description='σy [m]:',
                             continuous_update=False)
    sigxp_w = FloatLogSlider(value=1e-4, base=10, min=-5, max=-2,
                             step=0.01, description="σx' [rad]:",
                             continuous_update=False)
    sigyp_w = FloatLogSlider(value=1e-4, base=10, min=-5, max=-2,
                             step=0.01, description="σy' [rad]:",
                             continuous_update=False)
    slopex_w = FloatSlider(value=0.0, min=-10, max=10, step=0.1,
                           description='slope x [1/m]:',
                           continuous_update=False)
    slopey_w = FloatSlider(value=0.0, min=-10, max=10, step=0.1,
                           description='slope y [1/m]:',
                           continuous_update=False)

    # Initial view ranges – light → continuous_update=True
    xr_init_w  = FloatSlider(value=3.0, min=0.5, max=20, step=0.5,
                             description='x range [mm]:',
                             continuous_update=True)
    yr_init_w  = FloatSlider(value=3.0, min=0.5, max=20, step=0.5,
                             description='y range [mm]:',
                             continuous_update=True)
    xpr_init_w = FloatSlider(value=3.0, min=0.5, max=20, step=0.5,
                             description="x' range [mrad]:",
                             continuous_update=True)
    ypr_init_w = FloatSlider(value=3.0, min=0.5, max=20, step=0.5,
                             description="y' range [mrad]:",
                             continuous_update=True)

    zmax_w = FloatText(value=z_max_default, description='z_max [m]:',
                       layout=Layout(width='150px'))
    energy_w = FloatSlider(value=energy_default, min=1, max=200, step=1,
                           description='E [MeV]:',
                           continuous_update=False)

    add_q_btn = Button(description='Add Quad', button_style='success',
                       layout=Layout(width='100px'))
    del_q_btn = Button(description='Remove Quad', button_style='danger',
                       layout=Layout(width='110px'))

    def quad_labels():
        return [f"Q{i}: z={q['z']:.2f} m" for i, q in enumerate(quads)]

    quad_sel_w = Dropdown(options=quad_labels(),
                          value=quad_labels()[0],
                          description='Select quad:',
                          layout=Layout(width='200px'))

    # Quad strength / position – heavy → continuous_update=False
    G_w = FloatSlider(value=1.0, min=-20, max=20, step=0.1,
                      description='G [T/m]:',
                      continuous_update=True)
    zq_w = FloatSlider(value=1.0, min=0.1+LQ_EFFECTIVE/2,
                       max=z_max_default-LQ_EFFECTIVE/2-0.1,
                       step=0.01, description='z [m]:',
                       continuous_update=True)

    # View sliders – light → continuous_update=True
    zview_w = FloatSlider(value=1.0, min=0, max=z_max_default, step=0.01,
                          description='view z [m]:',
                          continuous_update=True)
    yscale_w = FloatSlider(value=1.0, min=0.5, max=10, step=0.1,
                           description='Y-scale:',
                           continuous_update=True)

    xr_cs_w = FloatSlider(value=5.0, min=0.5, max=50, step=0.5,
                          description='x img [mm]:',
                          continuous_update=True)
    yr_cs_w = FloatSlider(value=5.0, min=0.5, max=50, step=0.5,
                          description='y img [mm]:',
                          continuous_update=True)

    # ---- state ----
    x0 = xp0 = y0 = yp0 = None
    Sigma0_x = Sigma0_y = None
    z_grid = sigx_z = sigy_z = segments = None

    # ---- utilities ----
    def sel_index():
        label = quad_sel_w.value
        return int(label.split(':')[0][1:])

    def clamp_quad_positions():
        margin = 0.05
        min_spacing = LQ_EFFECTIVE + margin
        sorted_pairs = sorted(enumerate(quads), key=lambda iq: iq[1]['z'])

        for i in range(1, len(sorted_pairs)):
            idx_prev, q_prev = sorted_pairs[i-1]
            idx_cur,  q_cur  = sorted_pairs[i]
            if q_cur['z'] - q_prev['z'] < min_spacing:
                q_cur['z'] = q_prev['z'] + min_spacing

        zmax = float(zmax_w.value)
        zmin = LQ_EFFECTIVE/2 + 0.1
        zmaxc = zmax - LQ_EFFECTIVE/2 - 0.1
        for q in quads:
            q['z'] = min(max(q['z'], zmin), zmaxc)

    # ---- redraw functions ----
    def redraw_init():
        """Redraw ONLY initial distribution panel."""
        if x0 is None:
            return
        update_initial_distribution_plots(
            axs_init, x0, xp0, y0, yp0,
            xr_init_w.value, yr_init_w.value,
            xpr_init_w.value, ypr_init_w.value
        )
        with out_init:
            out_init.clear_output(wait=True)
            display(fig_init)

    def redraw_beam():
        """Redraw ONLY envelope + quads + x-y cross-section."""
        if x0 is None or z_grid is None or sigx_z is None or sigy_z is None:
            return

        zmax = float(zmax_w.value)
        draw_envelope_and_quads(
            ax_env, z_grid, sigx_z, sigy_z,
            quads, zmax, sel_index(),
            z_view=zview_w.value,
            yscale_factor=yscale_w.value
        )

        if segments is not None:
            xz, xpz, yz, ypz = propagate_particles_to_z(
                x0, xp0, y0, yp0, segments, zview_w.value
            )
            # subsample for plotting
            n = len(xz)
            if n > N_plot_xsec:
                idx = np.random.choice(n, size=N_plot_xsec, replace=False)
                x_plot = xz[idx]; y_plot = yz[idx]
            else:
                x_plot = xz; y_plot = yz

            draw_cross_section(
                ax_cross, x_plot, y_plot,
                xr_cs_w.value, yr_cs_w.value
            )

        with out_beam:
            out_beam.clear_output(wait=True)
            display(fig_beam)

    def redraw_all():
        """Update both panels (used when initial params change)."""
        redraw_init()
        redraw_beam()

    # ---- recompute functions ----
    def recompute_initial_and_lattice():
        nonlocal x0, xp0, y0, yp0, Sigma0_x, Sigma0_y
        x0, xp0, y0, yp0 = generate_initial_distribution(N_phys, init_params)
        Sigma0_x = sigma_from_particles(x0, xp0)
        Sigma0_y = sigma_from_particles(y0, yp0)
        redraw_init()
        recompute_lattice_and_envelope()

    def recompute_lattice_and_envelope():
        nonlocal z_grid, sigx_z, sigy_z, segments
        zmax = float(zmax_w.value)
        energy = energy_w.value

        quadsK = [{'z': q['z'], 'K': gradient_to_K(q['G'], energy)} for q in quads]
        segments, z_grid = build_lattice(quadsK, zmax)

        if Sigma0_x is None or Sigma0_y is None:
            return

        sigx_z, _ = propagate_sigma_plane(segments, z_grid, Sigma0_x, signK=+1)
        sigy_z, _ = propagate_sigma_plane(segments, z_grid, Sigma0_y, signK=-1)
        redraw_beam()

    # ---- callbacks ----
    def on_initial_param(_):
        init_params['dist_type'] = dist_type_w.value
        init_params['sigx']   = sigx_w.value
        init_params['sigy']   = sigy_w.value
        init_params['sigxp']  = sigxp_w.value
        init_params['sigyp']  = sigyp_w.value
        init_params['slope_x']= slopex_w.value
        init_params['slope_y']= slopey_w.value
        recompute_initial_and_lattice()

    def on_add_quad(_):
        zmax = float(zmax_w.value)
        if quads:
            z_new = min(quads[-1]['z'] + 0.5, zmax - LQ_EFFECTIVE/2 - 0.5)
        else:
            z_new = 1.0
        quads.append({'z': z_new, 'G': 1.0})
        clamp_quad_positions()
        quad_sel_w.options = quad_labels()
        quad_sel_w.value = quad_labels()[-1]
        on_quad_sel(None)

    def on_del_quad(_):
        if len(quads) <= 1:
            return
        quads.pop(sel_index())
        quad_sel_w.options = quad_labels()
        quad_sel_w.value = quad_labels()[0]
        on_quad_sel(None)

    def on_quad_sel(_):
        idx = sel_index()
        G_w.value = quads[idx]['G']
        zq_w.value = quads[idx]['z']
        redraw_beam()

    def on_G_change(_):
        quads[sel_index()]['G'] = G_w.value
        recompute_lattice_and_envelope()

    def on_zq_change(_):
        quads[sel_index()]['z'] = zq_w.value
        clamp_quad_positions()
        recompute_lattice_and_envelope()

    def on_zmax_change(_):
        zmax = float(zmax_w.value)
        if zmax <= LQ_EFFECTIVE + 0.5:
            zmax_w.value = LQ_EFFECTIVE + 1.0
            zmax = zmax_w.value
        zview_w.max = zmax
        zq_w.max = zmax - LQ_EFFECTIVE/2 - 0.1
        clamp_quad_positions()
        recompute_lattice_and_envelope()

    def on_energy_change(_):
        recompute_lattice_and_envelope()

    def on_init_view_change(_):
        # just axis limits in initial plots
        redraw_init()

    def simple_redraw_beam(_):
        # view z / y-scale / x-y ranges
        redraw_beam()

    # attach callbacks
    for w in [dist_type_w, sigx_w, sigy_w, sigxp_w, sigyp_w, slopex_w, slopey_w]:
        w.observe(on_initial_param, names='value')

    add_q_btn.on_click(on_add_quad)
    del_q_btn.on_click(on_del_quad)
    quad_sel_w.observe(on_quad_sel, names='value')
    G_w.observe(on_G_change, names='value')
    zq_w.observe(on_zq_change, names='value')
    zmax_w.observe(on_zmax_change, names='value')
    energy_w.observe(on_energy_change, names='value')

    for w in [xr_init_w, yr_init_w, xpr_init_w, ypr_init_w]:
        w.observe(on_init_view_change, names='value')

    for w in [zview_w, yscale_w, xr_cs_w, yr_cs_w]:
        w.observe(simple_redraw_beam, names='value')

    # layout
    left = VBox([
        dist_type_w, sigx_w, sigy_w, sigxp_w, sigyp_w,
        slopex_w, slopey_w,
        xr_init_w, yr_init_w, xpr_init_w, ypr_init_w
    ])
    top_row = HBox([left, out_init])

    quad_box = VBox([
        HBox([add_q_btn, del_q_btn]),
        quad_sel_w, G_w, zq_w,
        energy_w, zmax_w, zview_w, yscale_w,
        xr_cs_w, yr_cs_w
    ])
    bottom_row = HBox([quad_box, out_beam])

    ui = VBox([top_row, bottom_row])
    display(ui)

    # initialize everything
    on_initial_param(None)


In [ ]:
launch_toybox()